### Some useful references:
1. **[Training]**: https://github.com/LIHANG-HONG/birdclef2023-2nd-place-solution
2. **[Inference]**: https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-inference-birdclef-25

This model backbone is seresnext26t_32x4d

## SED model

In [34]:
MODE = "inference"  
KAGGLE_NOTEBOOK = False

In [35]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import joblib

import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from soundfile import SoundFile 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm.auto import tqdm
from glob import glob
import torchaudio
import random
import itertools
from typing import Union

import concurrent.futures

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [36]:
class CFG:
    def __init__(self, mode="inference", kaggle_notebook=False):
        self.seed = 42
        self.print_freq = 100
        self.num_workers = 4
        self.stage = "train_bce"
        self.mode = mode  # "train" or "inference"

        if kaggle_notebook:
        # ===== Path Settings =====
            self.train_datadir = "/kaggle/input/birdclef-2025/train_audio"
            self.train_csv = "/kaggle/input/birdclef-2025/train.csv"
            self.test_soundscapes = "/kaggle/input/birdclef-2025/test_soundscapes"
            self.submission_csv = "/kaggle/input/birdclef-2025/sample_submission.csv"
            self.taxonomy_csv = "/kaggle/input/birdclef-2025/taxonomy.csv"
            self.model_files = ["/kaggle/input/bird2025-sed-ckpt/sedmodel.pth"]
            
        else:
            self.train_datadir = "/kaggle/input/birdclef-2025/train_audio"
            self.train_csv = "/kaggle/input/birdclef-2025/train.csv"
            self.test_soundscapes = "../data/raw/test_soundscapes_small//"
            self.submission_csv = "../data/raw/sample_submission.csv"
            self.taxonomy_csv = "../data/raw/taxonomy.csv"
            self.model_files = ["../models/ensmbl_0527/bird2025-sed-ckpt/sedmodel.pth"]
            

        # ===== Model Settings =====
        self.model_name = "seresnext26t_32x4d"
        self.pretrained = False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.SR = 32000
        self.target_duration = 5
        self.train_duration = 10
        
        self.n_mels = 128
        self.n_fft = 2048
        self.hop_length = 512
        self.f_min = 20
        self.f_max = 16000
        self.normal = 80  # or 255
        self.infer_duration = 5
        self.duration_train = 10

        # ===== Device =====
        self.device = "cpu"

cfg = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)

In [37]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cpu
Loading taxonomy data...
Number of classes: 206


In [38]:
def set_seed(seed=42):
    """
    Set seed for reproducibility
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)


In [39]:
class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)


def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0)

def init_bn(bn):
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)

In [40]:

class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        self.num_classes = len(taxonomy_df)

        self.bn0 = nn.BatchNorm2d(cfg.n_mels)
        
        self.backbone = timm.create_model(
            cfg.model_name,
            pretrained=False,
            in_chans=cfg.in_channels,
            drop_rate=0.2,
            drop_path_rate=0.2,
        )

        layers = list(self.backbone.children())[:-2]
        self.encoder = nn.Sequential(*layers)
        
        if "efficientnet" in cfg.model_name:
            backbone_out = self.backbone.classifier.in_features
        elif "eca" in cfg.model_name:
            backbone_out = self.backbone.head.fc.in_features
        elif "res" in cfg.model_name:
            backbone_out = self.backbone.fc.in_features
        else:
            backbone_out = self.backbone.num_features
            
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
        self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="sigmoid")

        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=cfg.SR,
            hop_length=cfg.hop_length,
            n_mels=cfg.n_mels,
            f_min=cfg.f_min,
            f_max=cfg.f_max,
            n_fft=cfg.n_fft,
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        if cfg.device == "cuda":
            self.melspec_transform = self.melspec_transform.cuda()
        else:
            self.melspec_transform = self.melspec_transform.cpu()

        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=80
        )

    def extract_feature(self, x):
        x = x.permute((0, 1, 3, 2))
        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        x = x.transpose(2, 3)
        x = self.encoder(x)
        
        x = torch.mean(x, dim=2)
        
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        return x, frames_num
        
    @torch.cuda.amp.autocast(enabled=False)
    def transform_to_spec(self, audio):
        audio = audio.float()
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)

        if self.cfg.normal == 80:
            spec = (spec + 80) / 80
        elif self.cfg.normal == 255:
            spec = spec / 255
        else:
            raise NotImplementedError
                
        if self.cfg.in_channels == 3:
            spec = image_delta(spec)
        
        return spec

    def forward(self, x):
        with torch.no_grad():
            x = self.transform_to_spec(x)

        x, frames_num = self.extract_feature(x)
        
        clipwise_output, norm_att, segmentwise_output = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        return torch.logit(clipwise_output)

    def infer(self, x, tta_delta=2):
        with torch.no_grad():
            x = self.transform_to_spec(x)
        x, _ = self.extract_feature(x)
        time_att = torch.tanh(self.att_block.att(x))
        feat_time = x.size(-1)

        start = (
            feat_time / 2 - feat_time * (self.cfg.infer_duration / self.cfg.duration_train) / 2
        )
        end = start + feat_time * (self.cfg.infer_duration / self.cfg.duration_train)
        start = int(start)
        end = int(end)
        pred = self.attention_infer(start, end, x, time_att)

        start_minus = max(0, start - tta_delta)
        end_minus = end - tta_delta
        pred_minus = self.attention_infer(start_minus, end_minus, x, time_att)

        start_plus = start + tta_delta
        end_plus = min(feat_time, end + tta_delta)
        pred_plus = self.attention_infer(start_plus, end_plus, x, time_att)

        pred = 0.5 * pred + 0.25 * pred_minus + 0.25 * pred_plus
        return pred
        
    def attention_infer(self, start, end, x, time_att):
        feat = x[:, :, start:end]
        framewise_pred = torch.sigmoid(self.att_block.cla(feat))
        framewise_pred_max = framewise_pred.max(dim=2)[0]
        return framewise_pred_max

In [41]:
def load_sample(path, cfg):
    audio, orig_sr = sf.read(path, dtype="float32")
    seconds = []
    audio_length = cfg.SR * cfg.target_duration
    step = audio_length
    for i in range(audio_length, len(audio) + step, step):
        start = max(0, i - audio_length)
        end = start + audio_length
        if end > len(audio):
            pass
        else:
            seconds.append(int(end/cfg.SR))

    audio = np.concatenate([audio,audio,audio])
    audios = []
    for i,second in enumerate(seconds):
        end_seconds = int(second)
        start_seconds = int(end_seconds - cfg.target_duration)
    
        end_index = int(cfg.SR * (end_seconds + (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        start_index = int(cfg.SR * (start_seconds - (cfg.train_duration - cfg.target_duration) / 2) ) + len(audio) // 3
        end_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        start_pad = int(cfg.SR * (cfg.train_duration - cfg.target_duration) / 2) 
        y = audio[start_index:end_index].astype(np.float32)
        if i==0:
            y[:start_pad] = 0
        elif i==(len(seconds)-1):
            y[-end_pad:] = 0
        audios.append(y)

    return audios

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [42]:
def load_models(cfg, num_classes):
    """
    Load all model checkpoints from cfg.model_files and return initialized models
    """
    models = []
    model_files = cfg.model_files

    if not model_files:
        print(f"⚠️ No model files found under {cfg.model_path}!")
        return models

    print(f"🔍 Found {len(model_files)} model file(s).")

    for i, model_path in enumerate(model_files):
        try:
            print(f"📦 Loading model: {model_path}")

            checkpoint = torch.load(model_path, map_location=torch.device(cfg.device), weights_only=False)

            # checkpoint["cfg"] は dict の場合を想定して再構築
            if isinstance(checkpoint.get("cfg"), dict):
                cfg_dict = checkpoint["cfg"]
                cfg_temp = CFG()
                for k, v in cfg_dict.items():
                    setattr(cfg_temp, k, v)
            else:
                cfg_temp = checkpoint["cfg"]  # すでにCFGならそのまま使う

            cfg_temp.device = cfg.device  # 推論環境にあわせて上書き
            cfg_temp.taxonomy_csv = cfg.taxonomy_csv 

            model = BirdCLEFModel(cfg_temp)
            model.load_state_dict(checkpoint["model_state_dict"])
            
            model = model.to(cfg.device)
            model.eval()
            model.zero_grad()
            model.half().float()

            models.append(model)

        except Exception as e:
            print(f"❌ Error loading model {model_path}: {e}")

    return models

def predict_on_spectrogram(audio_path, models, cfg, species_ids):
    """Process a single audio file and predict species presence for each 5-second segment"""
    audio_path = str(audio_path)
    predictions = []
    row_ids = []
    soundscape_id = Path(audio_path).stem

    print(f"Processing {soundscape_id}")
    audio_data = load_sample(audio_path, cfg)
    for segment_idx, audio_input in enumerate(audio_data):
        
        end_time_sec = (segment_idx + 1) * cfg.target_duration
        row_id = f"{soundscape_id}_{end_time_sec}"
        row_ids.append(row_id)
        
        mel_spec = torch.tensor(audio_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        mel_spec = mel_spec.to(cfg.device)
        
        if len(models) == 1:
            with torch.no_grad():
                outputs = models[0].infer(mel_spec)
                final_preds = outputs.squeeze()
                # final_preds = torch.sigmoid(outputs).cpu().numpy().squeeze()

        else:
            segment_preds = []
            for model in models:
                with torch.no_grad():
                    outputs = model.infer(mel_spec)
                    probs = outputs.squeeze()
                    # probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                    segment_preds.append(probs)

            
            final_preds = np.mean(segment_preds, axis=0)
                
        predictions.append(final_preds)

    predictions = np.stack(predictions,axis=0)
    
    return row_ids, predictions

In [43]:
def run_inference(cfg, models, species_ids):
    """Run inference on all test soundscapes"""
    test_files = sorted(Path(cfg.test_soundscapes).glob('*.ogg'))
    
    print(f"Found {len(test_files)} test soundscapes")

    all_row_ids = []
    all_predictions = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        results = list(
            executor.map(
                predict_on_spectrogram,
                test_files,
                itertools.repeat(models),
                itertools.repeat(cfg),
                itertools.repeat(species_ids)
            )
        )

    for rids, preds in results:
        all_row_ids.extend(rids)
        all_predictions.extend(preds)
    
    return all_row_ids, all_predictions

def create_submission(row_ids, predictions, species_ids, cfg):
    """Create submission dataframe"""
    print("Creating submission dataframe...")

    submission_dict = {'row_id': row_ids}
    
    for i, species in enumerate(species_ids):
        submission_dict[species] = [pred[i] for pred in predictions]

    submission_df = pd.DataFrame(submission_dict)

    submission_df.set_index('row_id', inplace=True)

    sample_sub = pd.read_csv(cfg.submission_csv, index_col='row_id')

    missing_cols = set(sample_sub.columns) - set(submission_df.columns)
    if missing_cols:
        print(f"Warning: Missing {len(missing_cols)} species columns in submission")
        for col in missing_cols:
            submission_df[col] = 0.0

    submission_df = submission_df[sample_sub.columns]

    submission_df = submission_df.reset_index()
    
    return submission_df


def smooth_submission(submission_path):
        """
        Post-process the submission CSV by smoothing predictions to enforce temporal consistency.
        
        For each soundscape (grouped by the file name part of 'row_id'), each row's predictions
        are averaged with those of its neighbors using defined weights.
        
        :param submission_path: Path to the submission CSV file.
        """
        print("Smoothing submission predictions...")
        sub = pd.read_csv(submission_path)
        cols = sub.columns[1:]
        # Extract group names by splitting row_id on the last underscore
        groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
        unique_groups = np.unique(groups)
        
        for group in unique_groups:
            # Get indices for the current group
            idx = np.where(groups == group)[0]
            sub_group = sub.iloc[idx].copy()
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()
            
            if predictions.shape[0] > 1:
                # Smooth the predictions using neighboring segments
                new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)
                new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)
                for i in range(1, predictions.shape[0]-1):
                    new_predictions[i] = (predictions[i-1] * 0.2) + (predictions[i] * 0.6) + (predictions[i+1] * 0.2)
            # Replace the smoothed values in the submission dataframe
            sub.iloc[idx, 1:] = new_predictions
        
        sub.to_csv(submission_path, index=False)
        print(f"Smoothed submission saved to {submission_path}")

In [44]:

start_time = time.time()
print("Starting BirdCLEF-2025 inference...")

models = load_models(cfg, num_classes)

if not models:
    print("No models found! Please check model paths.")
    raise RuntimeError("No models loaded for inference.")

print(f"Model usage: {'Single model' if len(models) == 1 else f'Ensemble of {len(models)} models'}")

row_ids, predictions = run_inference(cfg, models, species_ids)

sub_sed_df = create_submission(row_ids, predictions, species_ids, cfg)


end_time = time.time()
print(f"Inference completed in {(end_time - start_time)/60:.2f} minutes")

Starting BirdCLEF-2025 inference...
🔍 Found 1 model file(s).
📦 Loading model: ../models/ensmbl_0527/bird2025-sed-ckpt/sedmodel.pth


Model usage: Single model
Found 4 test soundscapes
Processing H02_20230420_074000
Processing H02_20230420_112000
Processing H02_20230420_154500
Processing H02_20230502_080500
Creating submission dataframe...
Inference completed in 0.14 minutes


In [45]:
sub_sed_df.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.019034,0.014293,0.009067,0.003697,0.010040,0.009429,0.008441,0.013995,0.016443,...,0.007549,0.007677,0.010365,0.016833,0.007334,0.010245,0.009943,0.013151,0.020206,0.004325
1,H02_20230420_074000_10,0.005526,0.005460,0.003589,0.009103,0.005507,0.011483,0.009386,0.006968,0.010645,...,0.007237,0.005919,0.006159,0.010452,0.005051,0.012963,0.006981,0.013079,0.007854,0.008929
2,H02_20230420_074000_15,0.009316,0.016221,0.007963,0.011094,0.010349,0.008795,0.008718,0.010356,0.014014,...,0.009298,0.007677,0.009697,0.014609,0.006194,0.015556,0.010150,0.014794,0.012863,0.011697
3,H02_20230420_074000_20,0.010347,0.015713,0.004209,0.009343,0.012749,0.009912,0.008312,0.011245,0.014436,...,0.009790,0.007069,0.006696,0.016567,0.007894,0.015836,0.011767,0.014757,0.012819,0.010654
4,H02_20230420_074000_25,0.005793,0.005728,0.005349,0.011468,0.007462,0.011525,0.009423,0.006088,0.009496,...,0.008974,0.007244,0.006177,0.018225,0.007333,0.015601,0.011264,0.013051,0.012356,0.011748


## EfficientNet b0

In [46]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)
import hashlib
import json

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = cfg.test_soundscapes
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            
            # kaggle notebookならここを変更
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = cfg.test_soundscapes
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            
            # ローカルならここを変更
            self.model_path =  None
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = None # 下で指定する．
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            self.ensemble_strategy = "mean" # "mean", "max", "min", "median" など
            
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]
            



cfg = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import utils_lib, preprocess_lib

# Set seed
utils_lib.set_seed(cfg.seed)
# configをここで設定する．
def load_all_configs():
    cfg_list = []

    # sfzn1 hd hl512 psdMxp, 0.850
    cfg1 = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)
    cfg1.model_path = "../models/ensmbl_0527/bc25-models-fld0-sfzn1-hd-hl512-psd"
    cfg1.HOP_LENGTH = 512
    cfg_list.append(cfg1)

    # sfzn1 hd hl16 psdMxp, 0.849
    cfg2 = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)
    cfg2.HOP_LENGTH = 64
    cfg2.model_path = "../models/ensmbl_0527/bc25-models-fld0-sfzn1-hd-hl16-psdmxp"
    cfg_list.append(cfg2)
    
    # maxRMS psdMxp 0.843
    cfg3 = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)
    cfg3.model_path = "../models/ensmbl_0527/model-maxrms-0527-newjfcide/model_maxRMS_vino"
    cfg3.HOP_LENGTH = 64
    cfg_list.append(cfg3)
    
    # maxDB psdMxp 0.838
    cfg4 = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)
    cfg4.model_path = "../models/ensmbl_0527/model-maxdb-0527-newjfcide/model_maxdb_vino"
    cfg4.HOP_LENGTH = 64
    cfg_list.append(cfg4)
    
    # smart psdMxp 0.842
    cfg5 = CFG(mode=MODE, kaggle_notebook=KAGGLE_NOTEBOOK)
    cfg5.model_path = "../models/ensmbl_0527/model-sfzn1000-smart-pseudo-alpha0-2"
    cfg5.HOP_LENGTH = 64
    cfg_list.append(cfg5)
    
    return cfg_list


# 1. CFGオブジェクトからMel設定だけ抜き出してハッシュを作る
def cfg_to_mel_hash(cfg, exclude_keys=None):
    if exclude_keys is None:
        exclude_keys = [
            'mode', 'KAGGLE_NOTEBOOK', 'debug',
            'OUTPUT_DIR', 'train_datadir', 'train_csv', 'test_soundscapes',
            'submission_csv', 'taxonomy_csv', 'spectrogram_npy', 'MODELS_DIR',
            'model_path', 'device', 'batch_size', 'n_jobs',
            'use_tta', 'tta_count', 'threshold', 'use_specific_folds', 'folds',
            'debug_count', 'ensemble_strategy', 'epochs', 'selected_folds'
        ]
    
    cfg_dict = vars(cfg).copy()
    mel_cfg = {k: v for k, v in cfg_dict.items() if k not in exclude_keys}
    mel_cfg_str = json.dumps(mel_cfg, sort_keys=True)
    hash_val = hashlib.sha256(mel_cfg_str.encode()).hexdigest()[:8]
    return hash_val
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")
# mel変換
def process_audio_file(audio_path, cfg):
    """1ファイル分のmelspecデータを返す（row_id, melspecのリスト）"""
    dataset = []
    soundscape_id = Path(audio_path).stem
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))

        for segment_idx in range(total_segments):
            start = int(segment_idx * cfg.FS * cfg.WINDOW_SIZE)
            end = int(start + cfg.FS * cfg.WINDOW_SIZE)
            segment_audio = audio_data[start:end]

            mel_spec = preprocess_lib.process_audio_segment(segment_audio, cfg)
            row_id = f"{soundscape_id}_{(segment_idx + 1) * cfg.WINDOW_SIZE}"

            dataset.append({
                "row_id": row_id,
                "mel_spec": mel_spec
            })
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    return dataset


# 並列化してmelspecを生成
def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

# openvinoモデルの読み込み
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# openvinoモデルによる推論
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        # アンサンブル戦略の選択
        if cfg.ensemble_strategy == "mean":
            avg_preds = np.mean(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "max":
            avg_preds = np.max(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "min":
            avg_preds = np.min(preds_per_model, axis=0)
        elif cfg.ensemble_strategy == "median":
            avg_preds = np.median(preds_per_model, axis=0)
        else:
            raise ValueError(f"Unknown ensemble strategy: {cfg.ensemble_strategy}")

        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions
def run_inference(cfg, species_ids, dataset):

    print("Loading OpenVINO models...")
    vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
    models_ir = load_openvino_models(vino_dir, cfg)

    if not models_ir:
        raise RuntimeError("No OpenVINO models found.")

    print("Running OpenVINO inference...")
    if len(dataset) > 0:
        row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
    else:
        print("No test data available, generating empty submission.")
        row_ids = []
        predictions = []

    # smoothing前の予測値を保存
    submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)

    return submission_df
# 予測値のdf_listを受け取ってアンサンブル
def ensemble_submissions_dfs(dfs, method="mean"):
    """
    複数の submission DataFrame をアンサンブル

    Parameters:
        dfs (List[pd.DataFrame]): 各 submission.csv を読み込んだ DataFrame のリスト
        method (str): アンサンブル戦略（mean, max, median）

    Returns:
        pd.DataFrame: アンサンブル後の submission
    """
    assert all(df.columns[0] == "row_id" for df in dfs), "All DataFrames must start with 'row_id' column"
    row_ids = dfs[0]['row_id'].values
    preds = np.stack([df.iloc[:, 1:].values for df in dfs], axis=0)  # (n_models, n_rows, n_classes)

    if method == "mean":
        combined = np.mean(preds, axis=0)
    elif method == "max":
        combined = np.max(preds, axis=0)
    elif method == "median":
        combined = np.median(preds, axis=0)
    else:
        raise ValueError(f"Unsupported ensemble method: {method}")

    result_df = pd.DataFrame(combined, columns=dfs[0].columns[1:])
    result_df.insert(0, "row_id", row_ids)
    return result_df


def smooth_submission_df(submission_df, cfg, weights=None):
    """
    Smooth predictions using weighted moving average over a 5-frame window: [-2, -1, 0, +1, +2],
    then blend with per-class global average within each soundscape segment group.

    Parameters:
        submission_df: pd.DataFrame with 'row_id' and prediction columns.
        cfg: config object (interface compatibility).
        weights: List of 5 floats (default = [0.1, 0.2, 0.4, 0.2, 0.1]).

    Returns:
        pd.DataFrame with smoothed predictions.
    """
    print("Smoothing submission predictions with global average blend...")

    if weights is None:
        weights = np.array([0.1, 0.2, 0.4, 0.2, 0.1])
    else:
        weights = np.array(weights)

    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        preds = sub.iloc[idx][cols].values  # (T, C)
        T, C = preds.shape

        # エッジ処理：端を繰り返すようにパディング
        padded = np.pad(preds, ((2, 2), (0, 0)), mode='edge')  # (T+4, C)

        # 平滑化：5点加重平均（[-2, -1, 0, +1, +2]）
        smoothed = (
            padded[0:T]   * weights[0] +
            padded[1:T+1] * weights[1] +
            padded[2:T+2] * weights[2] +
            padded[3:T+3] * weights[3] +
            padded[4:T+4] * weights[4]
        )

        # 各クラスの平均予測を20%混ぜる（全セグメントに対して一様に加える）
        classwise_mean = smoothed.mean(axis=0, keepdims=True)  # shape: (1, C)
        smoothed = smoothed * 0.8 + classwise_mean * 0.2

        sub.iloc[idx, 1:] = smoothed

    return sub


Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [47]:
cfg_list = load_all_configs()

cfg_hash_list = []
for cfg in cfg_list:
    mel_hash = cfg_to_mel_hash(cfg)
    cfg_hash_list.append({
        "hash": mel_hash,
        "cfg": cfg
    })
print(cfg_hash_list)

 
datasets = {}
for item in cfg_hash_list:
    mel_hash = item["hash"]
    cfg = item["cfg"]

    if mel_hash not in datasets:
        print(f"🔵 Generating dataset for hash: {mel_hash}")
        datasets[mel_hash] = generate_melspec_dataset(cfg)


inference_dfs = []
for item in cfg_hash_list:
    mel_hash = item["hash"]
    cfg = item["cfg"]

    dataset = datasets[mel_hash]
    df = run_inference(cfg, species_ids, dataset)
    inference_dfs.append(df)



[{'hash': '12de2aa3', 'cfg': <__main__.CFG object at 0x7fe9c149cee0>}, {'hash': '0cae7500', 'cfg': <__main__.CFG object at 0x7fe9c149e8f0>}, {'hash': '0cae7500', 'cfg': <__main__.CFG object at 0x7fe9c149fc40>}, {'hash': '0cae7500', 'cfg': <__main__.CFG object at 0x7fe9c149ff10>}, {'hash': '0cae7500', 'cfg': <__main__.CFG object at 0x7fe9c149c5e0>}]
🔵 Generating dataset for hash: 12de2aa3


Parallel melspec gen:   0%|          | 0/4 [00:00<?, ?it/s]

🔵 Generating dataset for hash: 0cae7500


Parallel melspec gen:   0%|          | 0/4 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/ensmbl_0527/bc25-models-fld0-sfzn1-hd-hl512-psd
Running OpenVINO inference...
Creating submission dataframe...
Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/ensmbl_0527/bc25-models-fld0-sfzn1-hd-hl16-psdmxp
Running OpenVINO inference...
Creating submission dataframe...
Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/ensmbl_0527/model-maxrms-0527-newjfcide/model_maxRMS_vino
Running OpenVINO inference...
Creating submission dataframe...
Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/ensmbl_0527/model-maxdb-0527-newjfcide/model_maxdb_vino
Running OpenVINO inference...
Creating submission dataframe...
Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from .

In [48]:
# SEDのsubmissionを追加
inference_dfs.append(sub_sed_df)

In [49]:
# アンサンブル 何個しているか表示する
print(f"Ensembling {len(inference_dfs)} model predictions...")
ensemble_df = ensemble_submissions_dfs(inference_dfs, method="mean")
ensemble_df.to_csv(os.path.join(cfg_list[0].OUTPUT_DIR, 'submission_before_smoothing.csv'), index=False)
print("Saved ensembled (before smoothing) submission.")

# スムージング
smoothed_df = smooth_submission_df(ensemble_df, cfg_list[0])
smoothed_df.to_csv(os.path.join(cfg_list[0].OUTPUT_DIR, 'submission.csv'), index=False)
print("Saved smoothed final submission.")

Ensembling 6 model predictions...
Saved ensembled (before smoothing) submission.
Smoothing submission predictions with global average blend...
Saved smoothed final submission.


In [50]:
sub_sed_df.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.019034,0.014293,0.009067,0.003697,0.010040,0.009429,0.008441,0.013995,0.016443,...,0.007549,0.007677,0.010365,0.016833,0.007334,0.010245,0.009943,0.013151,0.020206,0.004325
1,H02_20230420_074000_10,0.005526,0.005460,0.003589,0.009103,0.005507,0.011483,0.009386,0.006968,0.010645,...,0.007237,0.005919,0.006159,0.010452,0.005051,0.012963,0.006981,0.013079,0.007854,0.008929
2,H02_20230420_074000_15,0.009316,0.016221,0.007963,0.011094,0.010349,0.008795,0.008718,0.010356,0.014014,...,0.009298,0.007677,0.009697,0.014609,0.006194,0.015556,0.010150,0.014794,0.012863,0.011697
3,H02_20230420_074000_20,0.010347,0.015713,0.004209,0.009343,0.012749,0.009912,0.008312,0.011245,0.014436,...,0.009790,0.007069,0.006696,0.016567,0.007894,0.015836,0.011767,0.014757,0.012819,0.010654
4,H02_20230420_074000_25,0.005793,0.005728,0.005349,0.011468,0.007462,0.011525,0.009423,0.006088,0.009496,...,0.008974,0.007244,0.006177,0.018225,0.007333,0.015601,0.011264,0.013051,0.012356,0.011748


In [51]:
inference_dfs[0].head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230502_080500_5,2.902312e-06,1.136564e-06,3.966986e-06,4.181187e-07,0.000011,1.136564e-06,0.000245,5.422219e-06,2.260324e-06,...,0.000296,0.000031,0.000140,0.000131,0.000090,0.000245,0.000048,0.000830,0.001501,0.000015
1,H02_20230502_080500_10,1.553498e-06,4.737905e-07,1.370957e-06,1.003014e-06,0.000003,8.315280e-07,0.000058,2.123378e-06,8.851569e-07,...,0.000404,0.000058,0.001099,0.000090,0.000191,0.000140,0.000040,0.003377,0.001284,0.000033
2,H02_20230502_080500_15,3.726639e-06,5.043474e-07,1.384600e-05,2.144495e-05,0.000006,9.516251e-06,0.000856,5.771920e-06,3.089494e-06,...,0.001455,0.000536,0.060975,0.000102,0.124213,0.000040,0.000006,0.000149,0.000080,0.000031
3,H02_20230502_080500_20,6.475948e-07,3.256312e-07,5.043474e-07,4.737905e-07,0.000002,2.873686e-07,0.000029,8.851569e-07,3.689885e-07,...,0.000217,0.000732,0.001245,0.000062,0.000779,0.000035,0.000007,0.000149,0.000296,0.000015
4,H02_20230502_080500_25,1.553498e-06,1.553498e-06,1.370957e-06,3.689885e-07,0.000007,3.689885e-07,0.000058,3.288749e-06,1.873875e-06,...,0.000296,0.001755,0.000033,0.000080,0.000335,0.000006,0.000109,0.000045,0.000346,0.000012


In [52]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head(12)


,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230502_080500_5,0.002345,0.002123,0.001210,0.001357,0.001470,0.001674,0.001528,0.001957,0.002416,...,0.001593,0.001386,0.003636,0.002973,0.003535,0.002062,0.001981,0.002481,0.003032,0.001189
1,H02_20230502_080500_10,0.001739,0.001932,0.000985,0.001675,0.001385,0.001727,0.001569,0.001685,0.002199,...,0.001648,0.001477,0.005139,0.002727,0.005383,0.002267,0.001934,0.002574,0.002491,0.001501
2,H02_20230502_080500_15,0.001547,0.002123,0.000989,0.001909,0.001513,0.001690,0.001607,0.001633,0.002168,...,0.001827,0.001733,0.008146,0.002814,0.009216,0.002466,0.002071,0.002523,0.002315,0.001766
3,H02_20230502_080500_20,0.001399,0.002027,0.000834,0.002214,0.001541,0.001761,0.001563,0.001546,0.002071,...,0.001767,0.001997,0.004956,0.003083,0.005642,0.002586,0.002361,0.002441,0.002230,0.001893
4,H02_20230502_080500_25,0.001211,0.001635,0.000782,0.002665,0.001310,0.001841,0.001566,0.001333,0.001850,...,0.001645,0.001924,0.003219,0.003403,0.003813,0.002625,0.002641,0.002274,0.002201,0.001988
5,H02_20230502_080500_30,0.001074,0.001378,0.000628,0.003359,0.001061,0.001960,0.001557,0.001185,0.001675,...,0.001468,0.001775,0.001836,0.003712,0.002157,0.002616,0.003412,0.002134,0.002088,0.002054
6,H02_20230502_080500_35,0.001024,0.001167,0.000536,0.003371,0.000877,0.002031,0.001495,0.001129,0.001604,...,0.001331,0.001643,0.002302,0.003663,0.002306,0.002529,0.003599,0.001988,0.002037,0.002073
7,H02_20230502_080500_40,0.001182,0.001236,0.000560,0.003147,0.000898,0.002178,0.001520,0.001303,0.001720,...,0.001287,0.001617,0.003124,0.003633,0.002577,0.002490,0.004275,0.002025,0.002073,0.002140
8,H02_20230502_080500_45,0.001398,0.001493,0.000648,0.003712,0.000979,0.002175,0.001559,0.001654,0.002028,...,0.001355,0.001584,0.003239,0.003389,0.002414,0.002592,0.003986,0.002259,0.002192,0.002251
9,H02_20230502_080500_50,0.001556,0.002199,0.000808,0.004347,0.001058,0.001943,0.001614,0.001927,0.002227,...,0.001455,0.001796,0.002475,0.003478,0.002106,0.002716,0.004540,0.002583,0.002229,0.002241


In [53]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head(12)


,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230502_080500_5,0.003176,0.002384,0.001514,0.000617,0.001692,0.001572,0.001463,0.002336,0.002750,...,0.001568,0.001304,0.001793,0.003148,0.001246,0.001787,0.001676,0.002409,0.003761,0.000725
1,H02_20230502_080500_10,0.000923,0.000912,0.000600,0.001518,0.000923,0.001914,0.001585,0.001165,0.001776,...,0.001381,0.001032,0.001693,0.001999,0.000894,0.002219,0.001213,0.002852,0.001858,0.001503
2,H02_20230502_080500_15,0.001556,0.002704,0.001342,0.001860,0.001728,0.001471,0.001797,0.001730,0.002337,...,0.002325,0.001618,0.020926,0.002533,0.024578,0.002621,0.001712,0.002580,0.002207,0.002008
3,H02_20230502_080500_20,0.001725,0.002620,0.000702,0.001558,0.002126,0.001653,0.001395,0.001876,0.002406,...,0.001866,0.002671,0.001648,0.002979,0.001716,0.002655,0.001983,0.002519,0.002360,0.001785
4,H02_20230502_080500_25,0.000967,0.000959,0.000895,0.001912,0.001246,0.001922,0.001582,0.001018,0.001584,...,0.001626,0.001969,0.001053,0.003344,0.001327,0.002603,0.002089,0.002204,0.002265,0.001964
5,H02_20230502_080500_30,0.000936,0.001300,0.000562,0.004639,0.000928,0.002085,0.001724,0.001104,0.001586,...,0.001485,0.001563,0.001110,0.004268,0.001860,0.002833,0.003883,0.002157,0.001990,0.002297
6,H02_20230502_080500_35,0.000721,0.000740,0.000336,0.003673,0.000600,0.001768,0.001313,0.000839,0.001416,...,0.001183,0.001573,0.000990,0.003527,0.001526,0.002597,0.002323,0.001818,0.001895,0.002051
7,H02_20230502_080500_40,0.000909,0.000682,0.000449,0.002962,0.000774,0.002517,0.001487,0.000756,0.001134,...,0.001068,0.001662,0.004176,0.004004,0.003001,0.002107,0.007428,0.001617,0.001820,0.001902
8,H02_20230502_080500_45,0.001581,0.001388,0.000453,0.002590,0.000959,0.002579,0.001540,0.001966,0.002312,...,0.001317,0.001308,0.004593,0.003195,0.002574,0.002531,0.002099,0.001966,0.002355,0.002577
9,H02_20230502_080500_50,0.001917,0.001851,0.001077,0.002584,0.001131,0.001765,0.001724,0.002496,0.002866,...,0.001470,0.001730,0.001471,0.003079,0.001045,0.003405,0.002118,0.003258,0.002498,0.002814


In [54]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (48, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', 'c

In [55]:
assert cfg.test_soundscapes == "/kaggle/input/birdclef-2025/test_soundscapes", "Test_soundscapes path is incorrect!"

AssertionError: Test_soundscapes path is incorrect!